In [1]:
import os
import requests
import json
import pandas as pd
import numpy as np
from datetime import datetime, date
import gmaps

# Import API key
from config import g_key

In [2]:
data_file = "Resources/Onyx-Data-UFO-Sightings-Dataset.csv"


In [3]:
ufoSightingFull_df = pd.read_csv(data_file,dtype={"datetime": "string", \
                                              "city": "string",\
                                              "state": "string",\
                                              "country": "string",\
                                              "shape":"string",\
                                              "duration":"string",\
                                              "duration (hours/min)":"string",\
                                              "comments":"string",\
                                              "date posted":"string",\
                                              "latitude":"string",\
                                              "longitude ":"string"})

C:\Users\Mary Mays\anaconda3\envs\AdvPandaEnv\lib\site-packages\IPython\core\interactiveshell.py:2714: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
ufoSightingFull_df.head()

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude
0,10/10/1949 20:30,san marcos,tx,us,cylinder,2700,45 minutes,This event took place in early fall around 194...,4/27/2004,29.8830556,-97.9411111
1,10/10/1949 21:00,lackland afb,tx,<NA>,light,7200,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.38421,-98.581082
2,10/10/1955 17:00,chester (uk/england),<NA>,gb,circle,20,20 seconds,Green/Orange circular disc over Chester&#44 En...,1/21/2008,53.2,-2.916667
3,10/10/1956 21:00,edna,tx,us,circle,20,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.9783333,-96.6458333
4,10/10/1960 20:00,kaneohe,hi,us,light,900,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.4180556,-157.8036111


In [5]:
#Lets pare this down - its a big dataset to handle clientside

In [6]:

ufoSightingFull_df.drop('date posted', axis=1,inplace = True)
ufoSighting_df = ufoSightingFull_df.sample(frac=0.50)


In [7]:
ufoSighting_df.count()
my_list = ufoSighting_df.columns.values.tolist()
my_list

['datetime',
 'city',
 'state',
 'country',
 'shape',
 'duration (seconds)',
 'duration (hours/min)',
 'comments',
 'latitude',
 'longitude ']

In [8]:
def convert_date(dt_tm):
    dt2 = datetime.strptime(dt_tm,"%m/%d/%Y %M:%S")
    return dt2.strftime("%m/%d/%Y")

ufoSighting_df['datetime'] = ufoSighting_df['datetime'].apply(convert_date)

In [9]:
ufoSighting_df['country'] = ufoSighting_df['country'].fillna('')
ufoSighting_df['state'] = ufoSighting_df['state'].fillna('')
ufoSighting_df.head()

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,latitude,longitude
28655,02/26/2005,ridgecrest,ca,us,unknown,1800,30 minutes,orange light drops what looks like asteroids,35.6225000,-117.6700000
33095,03/22/2013,toms river,nj,us,light,20,20 seconds,UFO dancing in the Toms River NJ sky takes off...,39.9536111,-74.1983333
56915,07/15/2000,vancouver,wa,us,circle,300,5 minutes,siting over vancouver&#44 Wa.,45.6388889,-122.6602778
43805,05/22/2000,gardner,ks,us,egg,20,20 seconds,This orange fireball suddenly slowed down and ...,38.8108333,-94.9269444
7820,11/10/2012,murrells inlet,sc,us,light,300,5 minutes,15-20 orange/reddish orbs&#44 moving in a simi...,33.5508333,-79.0416667


In [10]:
sightings_groups = ufoSighting_df.groupby(["city","country"])
sightings_groups

In [11]:
city_sightings_df = pd.DataFrame(columns= [
                               "city",
                               "longitude",
                               "latitude",
                               "count"])

for name,group in sightings_groups:
    if(len(group) > 20):
        city_loc = pd.Series({
                            "city": name[0],
                            "latitude" :group['latitude'].unique()[0],
                            "longitude":group['longitude '].unique()[0],
                            "count": len(group)})
        city_sightings_df = city_sightings_df.append(city_loc, ignore_index=True) 

In [12]:
city_sightings_df['latitude']

0      41.0813889
1      41.7894444
2      35.0844444
3      38.8047222
4      33.8352778
          ...    
259    40.1719444
260     49.883333
261    38.6786111
262    46.6022222
263    32.7252778
Name: latitude, Length: 264, dtype: object

In [13]:
gmaps.configure(api_key=g_key)
fig = gmaps.figure()

In [14]:
city_sightings_df['latitude'] = city_sightings_df['latitude'].astype(float, errors = 'raise')
city_sightings_df['longitude'] = city_sightings_df['longitude'].astype(float, errors = 'raise')
locations = city_sightings_df[["latitude", "longitude"]]


In [15]:
info_box_template = """
<dl>
<dt>City</dt><dd>{city}</dd>
<dt># of Sightings</dt><dd>{count}</dd>
</dl>
"""

city_info = [info_box_template.format(**row) for index, row in city_sightings_df.iterrows()]

In [16]:
markers = gmaps.marker_layer(locations,info_box_content=city_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))

In [17]:

json_data = ufoSighting_df.to_json(orient = "records")

In [18]:
json_split = json_data.split('},')
string_json = []
for entry in json_split:
    string = entry + "}" 
    string = string.replace("\\","")
    string = string.replace("[","")
    string = string.replace("]","")
    string = string.replace("}}","}")
    string_json.append(string)

In [19]:
print(string_json[len(string_json) -1])

{"datetime":"03/22/2000","city":"tuscaloosa","state":"al","country":"us","shape":"formation","duration (seconds)":"10","duration (hours/min)":"8-10 seconds","comments":"3 almost transparent&#44 almost circular (amoeba shaped)&#44 phosphorous neon-green entities flying in a triangular formation.  all 3 witness","latitude":"33.2097222","longitude ":"-87.5691667"}


In [20]:

with open('listfile.js', 'w') as filehandle:
    #add the bit in the f'ront
    filehandle.write('var data2 = [')
    for listitem in string_json:
        filehandle.write('%s,\n' % listitem)
    filehandle.write(']')